## Degree-corrected stochastic block model (DCSBM)

$$ f(A; \theta, B, \pi) = f(A | P) f(P) $$
$$ f(A | P) = \prod_i \prod_{i \neq j} P_{ij}^{A_{ij}} (1-P_{ij})^{1-A_{ij}} $$
$$ P_{ij} = \theta_i \theta_j B_{\tau_i,\tau_j}$$
$$ f(P_ij) = $$

new

$$ f(A|\theta, B, \tau) = \prod_{i \neq j} (\theta_i \theta_j B_{\tau_i,\tau_j})^{A_{ij}} (1 - \theta_i \theta_j B_{\tau_i,\tau_j} )^{1 - A_{ij}}$$

$$ f(A|\theta, B, \tau) = \prod_{i \neq j} (\theta_i \theta_j B_{\tau_i,\tau_j})^{A_{ij}} (1 - \theta_i \theta_j B_{\tau_i,\tau_j} )^{1 - A_{ij}}$$

$$ f(A|\theta, B, \pi) = f_A(A|\theta, B, \tau) f_T(\tau | \pi) $$


probability of observing these under the multinomial 
p(tau | pi) <- multinom